In [1]:
!git clone https://github.com/tatsukisato/kauto.git

Cloning into 'kauto'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 125 (delta 22), reused 119 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (125/125), 195.46 KiB | 7.82 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [2]:
# %cd /kaggle/working/kauto
# !git pull

In [3]:
# %cd ../

In [4]:
import sys
import os
from pathlib import Path
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm
from sklearn.metrics import f1_score
import numpy as np

In [5]:
def is_kaggle() -> bool:
    return "KAGGLE_KERNEL_RUN_TYPE" in os.environ

In [6]:
# Add project root to path to import src
if is_kaggle:
    sys.path.append(str(Path.cwd() / "kauto" / "competitions"))
else:
    sys.path.append(str(Path(__file__).resolve().parents[1]))

from src.utils import setup_directories, save_results, create_submission, print_experiment_info, crop_and_save_images
from src.image_dataset import ImageDataset
from src.cnn_model import SimpleCNN
from src.dataset import AtmaCup22Dataset

In [7]:
exp_name = "exp003_image_gpu"
description = "Image-based baseline with ResNet18 (Frozen backbone). Train on Q1, Val on Q2."
print_experiment_info(exp_name, description)

dirs = setup_directories(base_dir=Path("/kaggle/working/kauto/competitions"), data_dir=Path("/kaggle/input/atmacup22"))

In [8]:
# 1. Load Data
dataset_handler = AtmaCup22Dataset(data_dir=str(dirs['raw']))
train_meta, test_meta = dataset_handler.load_data()

Train data shape: (24920, 9)
Test data shape: (9223, 9)


In [9]:
# 2. Prepare Data Splitting
# Train: Q1, Val: Q2
# Filter by quarter string
train_df = train_meta[train_meta['quarter'].str.contains('Q1')].copy()
val_df = train_meta[train_meta['quarter'].str.contains('Q2')].copy()

print(f"Train set (Q1): {len(train_df)}")
print(f"Val set (Q2): {len(val_df)}")

Train set (Q1): 6410
Val set (Q2): 18510


In [10]:
# 3. Check/Generate Cropped Images
# We store generated crops in data/processed/crops_train
crops_dir = dirs['processed'] / 'crops_train'

# Check if crops exist for all training data (Q1+Q2)
# We use the length of original train_meta because indices are based on it
# and we want to ensure all potential images are processed if we change split later
if not crops_dir.exists() or len(list(crops_dir.glob("*.jpg"))) < len(train_meta) * 0.9: 
    # *0.9 allows for some missing/failed crops, but ideally should be full.
    # Let's just generate if dir doesn't exist or seems empty
    print(f"Generating cropped images to {crops_dir}...")
    crop_and_save_images(train_meta, dirs['raw'], crops_dir, mode='train')
else:
    print(f"Using existing cropped images in {crops_dir}")

Using existing cropped images in /kaggle/input/atmacup22/data/processed/crops_train


In [11]:
# 4. Transforms
# cv2 reads as numpy array (H, W, C). ToPILImage converts to PIL.
train_transform = transforms.Compose([
    transforms.ToPILImage(), 
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [12]:
# 5. Datasets & Loaders
train_dataset = ImageDataset(train_df, str(crops_dir), transform=train_transform, mode='train')
val_dataset = ImageDataset(val_df, str(crops_dir), transform=val_transform, mode='validation')

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [13]:
# 6. Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = SimpleCNN(num_classes=11, pretrained=True, freeze_backbone=True)
model.to(device)

Using device: cuda


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 189MB/s]


SimpleCNN(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_

In [14]:
# 7. Training Setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

epochs = 10
best_f1 = 0.0
# Create experiment specific output directory
exp_output_dir = dirs['output'] / exp_name
exp_output_dir.mkdir(parents=True, exist_ok=True)

# Update model path to be inside experiment directory
model_dir = exp_output_dir / 'models'
model_dir.mkdir(exist_ok=True)
best_model_path = model_dir / f"{exp_name}_best.pth"

In [15]:
# 8. Training Loop
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    
    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
    for images, labels in train_pbar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        train_pbar.set_postfix({'loss': loss.item()})
        
    train_loss /= len(train_dataset)
    
    # Validation
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())
    
    val_loss /= len(val_dataset)
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    
    print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Macro F1: {macro_f1:.4f}")
    
    scheduler.step(macro_f1)
    
    if macro_f1 > best_f1:
        best_f1 = macro_f1
        torch.save(model.state_dict(), best_model_path)
        print(f"New best model saved! F1: {best_f1:.4f}")

Epoch 1/10 [Val]: 100%|██████████| 579/579 [00:57<00:00, 10.10it/s]


Epoch 1: Train Loss: 1.7869, Val Loss: 2.3328, Val Macro F1: 0.3510
New best model saved! F1: 0.3510


Epoch 2/10 [Val]: 100%|██████████| 579/579 [00:38<00:00, 15.00it/s]


Epoch 2: Train Loss: 1.3044, Val Loss: 2.2705, Val Macro F1: 0.3931
New best model saved! F1: 0.3931


Epoch 3/10 [Val]: 100%|██████████| 579/579 [00:36<00:00, 15.65it/s]


...

Epoch 8: Train Loss: 0.9221, Val Loss: 2.2095, Val Macro F1: 0.4772
New best model saved! F1: 0.4772


Epoch 9/10 [Val]: 100%|██████████| 579/579 [00:37<00:00, 15.57it/s]


Epoch 9: Train Loss: 0.9190, Val Loss: 2.2323, Val Macro F1: 0.4742


Epoch 10/10 [Val]: 100%|██████████| 579/579 [00:36<00:00, 16.06it/s]

Epoch 10: Train Loss: 0.8974, Val Loss: 2.2976, Val Macro F1: 0.4727


In [16]:
# 9. Test Prediction
print("Starting prediction on Test Data (Q4)...")
if best_model_path.exists():
    model.load_state_dict(torch.load(best_model_path))
else:
    print("Warning: No best model found, using last epoch model")
    
model.eval()

# Test dataset
# For test, image_dir should be 'data/raw' as rel_paths are relative to it
test_dataset = ImageDataset(test_meta, str(dirs['raw']), transform=val_transform, mode='test')
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

test_preds = []
with torch.no_grad():
    for images in tqdm(test_loader, desc="Testing"):
        images = images.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        test_preds.extend(preds)

Starting prediction on Test Data (Q4)...


Testing: 100%|██████████| 289/289 [00:33<00:00,  8.51it/s]


In [17]:
# 10. Create Submission
sub_path = dirs['submissions'] / f"submission_{exp_name}.csv"
create_submission(test_preds, str(sub_path), test_meta)

# Save results
save_results({
    'best_val_f1': best_f1,
    'epochs': epochs,
    'config': {
        'backbone': 'resnet18',
        'img_size': 224,
        'batch_size': batch_size,
        'train_size': len(train_dataset),
        'val_size': len(val_dataset) 
    }
}, str(exp_output_dir), exp_name)

Submission saved to /kaggle/working/kauto/competitions/submissions/submission_exp003_image_gpu.csv
Submission shape: (9223, 1)
Label distribution:
label_id
0       72
1     1235
2     1180
3      845
4     1238
6     1083
7      892
8      692
9     1386
10     600
Name: count, dtype: int64
Results saved to /kaggle/working/kauto/competitions/output/exp003_image_gpu/exp003_image_gpu_results.json
